In [1]:
import pandas as pd
import numpy as np

In [2]:
data_path="../data/"
outputs_path="../outputs/"
test_set=pd.read_csv(outputs_path+"general/test_set_final.csv")
train_set=pd.read_csv(outputs_path+"general/train_set_final.csv")
dev_set=pd.read_csv(outputs_path+"general/dev_set_final.csv")

df_g=pd.read_csv("../sg_match/progress_singular_gpt3.csv")
low_ocurrence=pd.read_csv(data_path+"test_data/low_occurrence_annotated.csv")


In [ ]:
low_ocurrence=low_ocurrence.rename(columns={'Most Related SG goal':'goal1_raw',
       '(if exists) 2nd Related SG Goal':'goal2_raw', '(if exists) 3rd Related SG Goal':'goal3_raw'})
low_ocurrence=low_ocurrence.rename(columns={"SG_or_not":"label"})
low_ocurrence["label"]=low_ocurrence["label"].fillna(0)
low_ocurrence.abstract_clean=low_ocurrence.abstract_clean.fillna('')
low_ocurrence=low_ocurrence.assign(text=low_ocurrence.title_clean+". "+low_ocurrence.abstract_clean)
df_all_goals=pd.concat([dev_set,train_set,test_set,low_ocurrence])

In [2]:
def extract_predictions_gpt3(df):
    df=df.assign(sdg1=np.where(df['GPT3_response'].str.lower().str.contains("goal 1 |goal 1:|poverty"),1,0))
    df=df.assign(sdg2=np.where(df['GPT3_response'].str.lower().str.contains("goal 2|hunger"),1,0))
    df=df.assign(sdg3=np.where(df['GPT3_response'].str.lower().str.contains("goal 3|health"),1,0))
    df=df.assign(sdg4=np.where(df['GPT3_response'].str.lower().str.contains("goal 4|education"),1,0))
    df=df.assign(sdg5=np.where(df['GPT3_response'].str.lower().str.contains("goal 5|gender"),1,0))
    df=df.assign(sdg6=np.where(df['GPT3_response'].str.lower().str.contains("goal 6|clean water"),1,0))
    df=df.assign(sdg7=np.where(df['GPT3_response'].str.lower().str.contains("goal 7|clean energy"),1,0))
    df=df.assign(sdg8=np.where(df['GPT3_response'].str.lower().str.contains("goal 8|decent work"),1,0))
    df=df.assign(sdg9=np.where(df['GPT3_response'].str.lower().str.contains("goal 9|industry|innovation"),1,0))
    df=df.assign(sdg10=np.where(df['GPT3_response'].str.lower().str.contains("goal 10|inequal"),1,0))
    df=df.assign(sdg11=np.where(df['GPT3_response'].str.lower().str.contains("goal 11|sustainable cities"),1,0))
    df=df.assign(sdg12=np.where(df['GPT3_response'].str.lower().str.contains("goal 12|responsible consumption"),1,0))
    df=df.assign(sdg13=np.where(df['GPT3_response'].str.lower().str.contains("goal 13|climate"),1,0))
    df=df.assign(sdg14=np.where(df['GPT3_response'].str.lower().str.contains("goal 14|life below water"),1,0))
    df=df.assign(sdg15=np.where(df['GPT3_response'].str.lower().str.contains("goal 15|life on land"),1,0))
    df=df.assign(sdg16=np.where(df['GPT3_response'].str.lower().str.contains("goal 16|peace|justice"),1,0))
    df=df.assign(sdg17=np.where(df['GPT3_response'].str.lower().str.contains("goal 17|partnership"),1,0))
    return df

In [3]:
df_g=extract_predictions_gpt3(df_g)

NameError: name 'df_g' is not defined

In [6]:
mapping_sdg={'sdg1': 'No Poverty',
 'sdg2': 'Zero Hunger',
 'sdg3': 'Good Health and Well-Being',
 'sdg4': 'Quality Education',
 'sdg5': 'Gender Equality',
 'sdg6': 'Clean Water and Sanitation',
 'sdg7': 'Affordable and Clean Energy',
 'sdg8': 'Decent Work and Economic Growth',
 'sdg9': 'Industry, Innovation and Infrastructure',
 'sdg10': 'Reduced Inequalities',
 'sdg11': 'Sustainable Cities and Communities',
 'sdg12': 'Responsible Consumption and Production',
 'sdg13': 'Climate Action',
 'sdg14': 'Life Below Water',
 'sdg15': 'Life on Land',
 'sdg16': 'Peace, Justice and Strong Institutions',
 'sdg17': 'Partnership for the Goals'
}

In [7]:
df_g=pd.melt(df_g,id_vars=['ID','year'],value_vars=['sdg1', 'sdg2', 'sdg3', 'sdg4', 'sdg5',
       'sdg6', 'sdg7', 'sdg8', 'sdg9', 'sdg10', 'sdg11', 'sdg12', 'sdg13',
       'sdg14', 'sdg15', 'sdg16', 'sdg17'],value_name='goal_binary',var_name='goal_number')

In [8]:
df_g=df_g.loc[df_g.goal_binary==1]

In [9]:
df_g['Goal']=df_g.goal_number.replace(mapping_sdg)

In [10]:
df_all_goals.goal1_raw=df_all_goals.goal1_raw.fillna('')
df_all_goals.goal2_raw=df_all_goals.goal2_raw.fillna('')
df_all_goals.goal3_raw=df_all_goals.goal3_raw.fillna('')
df_all_goals=df_all_goals.assign(goal1=np.where(df_all_goals['goal1_raw'].str.lower().str.contains("education"),'Quality Education',
                    np.where(df_all_goals['goal1_raw'].str.lower().str.contains("poverty"),'No Poverty',
                    np.where(df_all_goals['goal1_raw'].str.lower().str.contains("hunger"),'Zero Hunger',
                    np.where(df_all_goals['goal1_raw'].str.lower().str.contains("clean_water"),'Clean Water and Sanitation',
                    np.where(df_all_goals['goal1_raw'].str.lower().str.contains("clean_energy"),'Affordable and Clean Energy',
                    np.where(df_all_goals['goal1_raw'].str.lower().str.contains("life_land"),'Life on Land',
                    np.where(df_all_goals['goal1_raw'].str.lower().str.contains("marine_life"),'Life Below Water',
                    np.where(df_all_goals['goal1_raw'].str.lower().str.contains("health"),'Good Health and Well-Being',
                    np.where(df_all_goals['goal1_raw'].str.lower().str.contains("climate"),'Climate Action',
                    np.where(df_all_goals['goal1_raw'].str.lower().str.contains("peace|privacy|disinformation_and_fake_news|deception|hate"),'Peace, Justice and Strong Institutions',
                    np.where(df_all_goals['goal1_raw'].str.lower().str.contains("social biases|race & identity"),'Reduced Inequalities',
                    np.where(df_all_goals['goal1_raw'].str.lower().str.contains("industry|innovation|research"),'Industry, Innovation and Infrastructure',
                    np.where(df_all_goals['goal1_raw'].str.lower().str.contains("sustainable cities|sustainable_cities"),'Sustainable Cities and Communities',
                    np.where(df_all_goals['goal1_raw'].str.lower().str.contains("gender"),'Gender Equality',
                    np.where(df_all_goals['goal1_raw'].str.lower().str.contains("decent work|decent_work_and_economy"),'Decent Work and Economic Growth',
                    np.where(df_all_goals['goal1_raw'].str.lower().str.contains("partnership"),'Partnership for the Goals',
                    np.where(df_all_goals['goal1_raw'].str.lower().str.contains("responsible_consumption_and_production"),'Responsible Consumption and Production',
                    np.where(df_all_goals['goal1_raw'].str.lower().str.contains("reduced|social_equality"),'Reduced Inequalities',''
                          )))))))))))))))))))

df_all_goals=df_all_goals.assign(goal2=np.where(df_all_goals['goal2_raw'].str.lower().str.contains("education"),'Quality Education',
                    np.where(df_all_goals['goal2_raw'].str.lower().str.contains("poverty"),'No Poverty',
                    np.where(df_all_goals['goal2_raw'].str.lower().str.contains("hunger"),'Zero Hunger',
                    np.where(df_all_goals['goal2_raw'].str.lower().str.contains("clean_water"),'Clean Water and Sanitation',
                    np.where(df_all_goals['goal2_raw'].str.lower().str.contains("clean_energy"),'Affordable and Clean Energy',
                    np.where(df_all_goals['goal2_raw'].str.lower().str.contains("life_land"),'Life on Land',
                    np.where(df_all_goals['goal2_raw'].str.lower().str.contains("marine_life"),'Life Below Water',         
                    np.where(df_all_goals['goal2_raw'].str.lower().str.contains("health"),'Good Health and Well-Being',
                    np.where(df_all_goals['goal2_raw'].str.lower().str.contains("climate"),'Climate Action',
                    np.where(df_all_goals['goal2_raw'].str.lower().str.contains("peace|privacy|disinformation_and_fake_news|deception|hate"),'Peace, Justice and Strong Institutions',
                    np.where(df_all_goals['goal2_raw'].str.lower().str.contains("social biases|race & identity"),'Reduced Inequalities',
                    np.where(df_all_goals['goal2_raw'].str.lower().str.contains("industry|innovation|research"),'Industry, Innovation and Infrastructure',
                    np.where(df_all_goals['goal2_raw'].str.lower().str.contains("sustainable cities|sustainable_cities"),'Sustainable Cities and Communities',
                    np.where(df_all_goals['goal2_raw'].str.lower().str.contains("gender"),'Gender Equality',
                    np.where(df_all_goals['goal2_raw'].str.lower().str.contains("decent work|decent_work_and_economy"),'Decent Work and Economic Growth',
                    np.where(df_all_goals['goal2_raw'].str.lower().str.contains("partnership"),'Partnership for the Goals',
                    np.where(df_all_goals['goal2_raw'].str.lower().str.contains("responsible_consumption_and_production"),'Responsible Consumption and Production',
                    np.where(df_all_goals['goal2_raw'].str.lower().str.contains("reduced|social_equality"),'Reduced Inequalities',''
                          )))))))))))))))))))

df_all_goals=df_all_goals.assign(goal3=np.where(df_all_goals['goal3_raw'].str.lower().str.contains("education"),'Quality Education',
                    np.where(df_all_goals['goal3_raw'].str.lower().str.contains("poverty"),'No Poverty',
                    np.where(df_all_goals['goal3_raw'].str.lower().str.contains("hunger"),'Zero Hunger',
                    np.where(df_all_goals['goal3_raw'].str.lower().str.contains("clean_water"),'Clean Water and Sanitation',
                    np.where(df_all_goals['goal3_raw'].str.lower().str.contains("clean_energy"),'Affordable and Clean Energy',
                    np.where(df_all_goals['goal3_raw'].str.lower().str.contains("life_land"),'Life on Land',
                    np.where(df_all_goals['goal3_raw'].str.lower().str.contains("marine_life"),'Life Below Water',
                    np.where(df_all_goals['goal3_raw'].str.lower().str.contains("health"),'Good Health and Well-Being',
                    np.where(df_all_goals['goal3_raw'].str.lower().str.contains("climate"),'Climate Action',
                    np.where(df_all_goals['goal3_raw'].str.lower().str.contains("peace|privacy|disinformation_and_fake_news|deception|hate"),'Peace, Justice and Strong Institutions',
                    np.where(df_all_goals['goal3_raw'].str.lower().str.contains("social biases|race & identity"),'Reduced Inequalities',
                    np.where(df_all_goals['goal3_raw'].str.lower().str.contains("industry|innovation|research"),'Industry, Innovation and Infrastructure',
                    np.where(df_all_goals['goal3_raw'].str.lower().str.contains("sustainable cities|sustainable_cities"),'Sustainable Cities and Communities',
                    np.where(df_all_goals['goal3_raw'].str.lower().str.contains("gender"),'Gender Equality',
                    np.where(df_all_goals['goal3_raw'].str.lower().str.contains("decent work|decent_work_and_economy"),'Decent Work and Economic Growth',
                    np.where(df_all_goals['goal3_raw'].str.lower().str.contains("partnership"),'Partnership for the Goals',
                    np.where(df_all_goals['goal3_raw'].str.lower().str.contains("responsible_consumption_and_production"),'Responsible Consumption and Production',
                    np.where(df_all_goals['goal3_raw'].str.lower().str.contains("reduced|social_equality"),'Reduced Inequalities',''
                          )))))))))))))))))))


In [11]:
df_group=pd.concat([df_all_goals.loc[:,['ID','goal1','year']],df_all_goals.loc[:,['ID','goal2','year']].rename(columns={'goal2':'goal1'}),df_all_goals.loc[:,['ID','goal3','year']].rename(columns={'goal3':'goal1'})])

In [12]:
df_group=df_group.loc[df_group.goal1!='']

In [13]:
df_group=df_group.rename(columns={'goal1':'Goal'})

In [14]:
df_papers=pd.concat([df_group,df_g]).reset_index(drop=True)

In [15]:
df_papers=df_papers.loc[~df_papers.year.isna()]

In [16]:
df_papers.year=df_papers.year.apply(int)

In [17]:
df_agg=df_papers.groupby(['Goal','year']).ID.count().reset_index()


df_agg=df_agg.assign(total=df_agg.groupby(['year']).ID.transform(lambda x:x.sum()))

df_agg=df_agg.assign(total_max=df_agg.groupby(['year']).ID.transform(lambda x:x.max()))

df_agg=df_agg.assign(proportion=df_agg.ID/df_agg.total)
df_agg=df_agg.assign(proportion_max=df_agg.ID/df_agg.total_max)

df_agg=df_agg.rename(columns={'ID':'papers'})

In [18]:
df_agg

,Goal,year,papers,total,total_max,proportion,proportion_max
0,Affordable and Clean Energy,2019,2,817,286,0.002448,0.006993
1,Affordable and Clean Energy,2020,1,1443,442,0.000693,0.002262
2,Affordable and Clean Energy,2021,1,1210,442,0.000826,0.002262
3,Clean Water and Sanitation,2017,1,430,157,0.002326,0.006369
4,Clean Water and Sanitation,2020,1,1443,442,0.000693,0.002262
...,...,...,...,...,...,...,...
363,Zero Hunger,2009,1,148,69,0.006757,0.014493
364,Zero Hunger,2012,1,312,97,0.003205,0.010309
365,Zero Hunger,2018,1,588,189,0.001701,0.005291
366,Zero Hunger,2019,2,817,286,0.002448,0.006993


## all years

In [19]:
df_agg_total=df_papers.groupby(['Goal']).ID.count().reset_index()
df_agg_total=df_agg_total.assign(total=df_agg_total.ID.sum())
df_agg_total=df_agg_total.assign(proportion=df_agg_total.ID/df_agg_total.total)
df_agg_total=df_agg_total.assign(total_max=df_agg_total.ID.max())
df_agg_total=df_agg_total.assign(proportion_max=df_agg_total.ID/df_agg_total.total_max)
df_agg_total=df_agg_total.rename(columns={'ID':'papers'})

df_agg_total=df_agg_total.assign(year="All years")

In [20]:
df_agg_final=pd.concat([df_agg,df_agg_total])

In [21]:
df_agg.loc[df_agg.year==2010]

,Goal,year,papers,total,total_max,proportion,proportion_max
7,Climate Action,2010,1,226,82,0.004425,0.012195
48,Decent Work and Economic Growth,2010,13,226,82,0.057522,0.158537
67,Gender Equality,2010,1,226,82,0.004425,0.012195
110,Good Health and Well-Being,2010,82,226,82,0.362832,1.000000
154,"Industry, Innovation and Infrastructure",2010,6,226,82,0.026549,0.073171
175,Life on Land,2010,1,226,82,0.004425,0.012195
191,Partnership for the Goals,2010,1,226,82,0.004425,0.012195
223,"Peace, Justice and Strong Institutions",2010,29,226,82,0.128319,0.353659
274,Quality Education,2010,62,226,82,0.274336,0.756098
304,Reduced Inequalities,2010,5,226,82,0.022124,0.060976


In [22]:
df_agg.loc[df_agg.year==2010]

,Goal,year,papers,total,total_max,proportion,proportion_max
7,Climate Action,2010,1,226,82,0.004425,0.012195
48,Decent Work and Economic Growth,2010,13,226,82,0.057522,0.158537
67,Gender Equality,2010,1,226,82,0.004425,0.012195
110,Good Health and Well-Being,2010,82,226,82,0.362832,1.000000
154,"Industry, Innovation and Infrastructure",2010,6,226,82,0.026549,0.073171
175,Life on Land,2010,1,226,82,0.004425,0.012195
191,Partnership for the Goals,2010,1,226,82,0.004425,0.012195
223,"Peace, Justice and Strong Institutions",2010,29,226,82,0.128319,0.353659
274,Quality Education,2010,62,226,82,0.274336,0.756098
304,Reduced Inequalities,2010,5,226,82,0.022124,0.060976


In [23]:
df_agg_final.Goal.value_counts()

Quality Education                          52
Industry, Innovation and Infrastructure    45
Decent Work and Economic Growth            44
Good Health and Well-Being                 44
Peace, Justice and Strong Institutions     40
Sustainable Cities and Communities         37
Reduced Inequalities                       31
Gender Equality                            20
Climate Action                             14
Partnership for the Goals                  14
Life on Land                                9
Zero Hunger                                 9
Responsible Consumption and Production      8
Life Below Water                            8
Affordable and Clean Energy                 4
No Poverty                                  3
Clean Water and Sanitation                  3
Name: Goal, dtype: int64

In [24]:
mapping_all={
'No Poverty':'G1. No Poverty',
'Zero Hunger':'G2. No Hunger',
'Good Health and Well-Being':'G3. Good Health and Well-Being',
'Quality Education':'G4. Quality Education',
'Gender Equality':'G5. Gender Equality',
'Clean Water and Sanitation':'G6. Clean Water and Sanitation',
'Affordable and Clean Energy':'G7. Affordable and Clean Energy',
'Decent Work and Economic Growth':'G8. Decent Work and Economic Growth',
'Industry, Innovation and Infrastructure':'G9. Industry, Innovation and Infrastructure',
'Industry, Innovation and Infrastrucure':'G9. Industry, Innovation and Infrastructure',
'Reduced Inequalities':'G10. Reduced Inequalities',
'Sustainable Cities and Communities':'G11. Sustainable Cities and Communities',
'Responsible Consumption and Production':'G12. Responsible Consumption and Production',
'Climate Action':'G13. Climate Action',
'Life Below Water':'G14. Life Below Water',
'Life on Land':'G15. Life on Land',
'Peace, Justice and Strong Institutions':'G16. Peace, Justice and Strong Institutions',
'Partnership for the Goals':'G17. Partnership for the Goals'
}


In [25]:
df_agg_final['goal_long']=df_agg_final.Goal.replace(mapping_all)

In [26]:
from itertools import product

In [27]:
df_dict=pd.DataFrame(list(product(list(df_agg_final.year.unique()), list(df_agg_final.Goal.unique()))), columns=['year', 'Goal'])

In [29]:
df_dict['goal_long']=df_dict.Goal.replace(mapping_all)

In [34]:
df_dict=df_dict.merge(df_agg_final,on=['year','Goal','goal_long'],how='left')

In [36]:
df_dict.papers=df_dict.papers.fillna(0)

In [54]:
mapping_all_n={
'No Poverty':'01',
'Zero Hunger':'02',
'Good Health and Well-Being':'03',
'Quality Education':'04',
'Gender Equality':'05',
'Clean Water and Sanitation':'06',
'Affordable and Clean Energy':'07',
'Decent Work and Economic Growth':'08',
'Industry, Innovation and Infrastructure':'09',
'Industry, Innovation and Infrastrucure':'09',
'Reduced Inequalities':'10',
'Sustainable Cities and Communities':'11',
'Responsible Consumption and Production':'12',
'Climate Action':'13',
'Life Below Water':'14',
'Life on Land':'15',
'Peace, Justice and Strong Institutions':'16',
'Partnership for the Goals':'17'
}


In [55]:
df_dict['goal_number']=df_dict.Goal.replace(mapping_all_n)

In [56]:
df_dict

,year,Goal,goal_long,papers,total,total_max,proportion,proportion_max,goal_number
179,2014,No Poverty,G1. No Poverty,0.0,NaN,NaN,NaN,NaN,01
944,1977,No Poverty,G1. No Poverty,0.0,NaN,NaN,NaN,NaN,01
400,1989,No Poverty,G1. No Poverty,0.0,NaN,NaN,NaN,NaN,01
94,1990,No Poverty,G1. No Poverty,0.0,NaN,NaN,NaN,NaN,01
349,1985,No Poverty,G1. No Poverty,0.0,NaN,NaN,NaN,NaN,01
...,...,...,...,...,...,...,...,...,...
537,1999,Partnership for the Goals,G17. Partnership for the Goals,0.0,NaN,NaN,NaN,NaN,17
588,2002,Partnership for the Goals,G17. Partnership for the Goals,0.0,NaN,NaN,NaN,NaN,17
316,1981,Partnership for the Goals,G17. Partnership for the Goals,0.0,NaN,NaN,NaN,NaN,17
282,1978,Partnership for the Goals,G17. Partnership for the Goals,0.0,NaN,NaN,NaN,NaN,17


df_agg_final.Goal=df_agg_final.Goal.replace({'Zero Hunger':'No Hunger'})

In [57]:
df_dict=df_dict.sort_values(['goal_number'])

In [58]:
df_dict

,year,Goal,goal_long,papers,total,total_max,proportion,proportion_max,goal_number
179,2014,No Poverty,G1. No Poverty,0.0,NaN,NaN,NaN,NaN,01
519,1998,No Poverty,G1. No Poverty,0.0,NaN,NaN,NaN,NaN,01
264,1976,No Poverty,G1. No Poverty,0.0,NaN,NaN,NaN,NaN,01
757,1969,No Poverty,G1. No Poverty,0.0,NaN,NaN,NaN,NaN,01
145,2012,No Poverty,G1. No Poverty,1.0,312.0,97.0,0.003205,0.010309,01
...,...,...,...,...,...,...,...,...,...
401,1989,Partnership for the Goals,G17. Partnership for the Goals,0.0,NaN,NaN,NaN,NaN,17
163,2013,Partnership for the Goals,G17. Partnership for the Goals,0.0,NaN,NaN,NaN,NaN,17
299,1980,Partnership for the Goals,G17. Partnership for the Goals,0.0,NaN,NaN,NaN,NaN,17
826,1991,Partnership for the Goals,G17. Partnership for the Goals,1.0,16.0,8.0,0.062500,0.125000,17


In [59]:
df_dict.to_json(outputs_path+"/sg_visualization/proportions_years.json",orient="records")